# Практическое задание 3

# Named Entity Recognition

## Введение

### Постановка задачи

В этом задании вы будете решать задачу извлечения именованных сущностей (Named Entity Recognition) - одну из самых распространенных в NLP наряду с задачей текстовой классификации.

Данная задача заключается в том, что нужно классифицировать каждое слово / токен на предмет того, является ли оно частью именованной сущности (сущность может состоять из нескольких слов / токенов) или нет.

Например, мы хотим извлечь имена и названия организаций. Тогда для текста

    Yan    Goodfellow  works  for  Google  Brain

модель должна извлечь следующую последовательность:

    B-PER  I-PER       O      O    B-ORG   I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система (*BIO*-разметка) введена, чтобы различать последовательные именованные сущности одного типа.
Существуют и другие типы разметок, например *BILUO*, но в рамках данного практического задания сфокусируемся имеено на *BIO*.

Решать NER задачу мы будем на датасете CoNLL-2003 с использованием рекуррентных сетей и моделей на базе архитектуры Transformer.

### Библиотеки

Основные библиотеки:
 - [PyTorch](https://pytorch.org/)
 - [Transformers](https://github.com/huggingface/transformers)
 
### Данные

Данные лежат в архиве, который состоит из:

- *train.tsv* - обучающая выборка. В каждой строке записаны: <слово / токен>, <тэг слова / токена>

- *valid.tsv* - валидационная выборка, которую можно использовать для подбора гиперпарамеров и замеров качества. Имеет идентичную с train.tsv структуру.

- *test.tsv* - тестовая выборка, по которой оценивается итоговое качество. Имеет идентичную с train.tsv структуру.

Скачать данные можно здесь: [ссылка](https://github.com/dayyass/msu_task_3_ner)

In [1]:
# !pip install numpy==1.21.6 scikit-learn==1.0.2 tensorboard==2.9.0 torch==1.12.1 tqdm==4.64.0 transformers==4.21.1

In [1]:
import random
from collections import Counter, defaultdict, namedtuple
from typing import Tuple, List, Dict, Any

import torch
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

Зафиксируем seed для воспроизводимости результатов (желательно делать **всегда**!):

In [2]:
def set_global_seed(seed: int) -> None:
    """
    Set global seed for reproducibility.
    """

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_global_seed(42)

Проинициализируем device (CPU / GPU) на котором будем работать (желательно **GPU**):

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

Здесь и далее проинициализируем *tensorboard* для логгирования метрики в процессе обучения:

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Launching TensorBoard...

## Часть 1. Подготовка данных (4 балла)

Первым делом нам нужно считать данные. Давайте напишем функцию, которая на вход принимает путь до одного из conll-2003 файла и возвращает два списка:
- список списков слов / токенов (и соответствующий ему)
- список списков тегов

P.S. Сделаем данную функцию более гибкой, подавая на вход еще булеву переменную, считываем ли мы данные в *lowercase* или нет.

**Задание. Реализуйте функцию read_conll2003.** **<font color='red'>(1 балл)</font>**

In [6]:
def read_conll2003(
    path: str,
    lower: bool = True,
) -> Tuple[List[List[str]], List[List[str]]]:
    """
    Prepare data in CoNNL like format.
    """
    with open(path, 'r') as f:
        tokens_labels = f.readlines()

    token_seq, label_seq = [], []
    cur_token, cur_label = [], []
    
    for token_label in tokens_labels:
        if token_label == '\n':
            token_seq.append(cur_token)
            label_seq.append(cur_label)
            cur_token, cur_label = [], []
        else:
            token, label = token_label.split()
            if lower:
                token = token.lower()
            cur_token.append(token)
            cur_label.append(label)
            
    # YOUR CODE HERE

    return token_seq, label_seq

Считаем все три файла:
- *train.tsv*
- *valid.tsv*
- *test.tsv*

In [7]:
train_token_seq, train_label_seq = read_conll2003("data/train.tsv")
valid_token_seq, valid_label_seq = read_conll2003("data/valid.tsv")
test_token_seq, test_label_seq = read_conll2003("data/test.tsv")

Посмотрим на то, что мы получили:

In [8]:
for token, label in zip(train_token_seq[0], train_label_seq[0]):
    print(f"{token}\t{label}")

eu	B-ORG
rejects	O
german	B-MISC
call	O
to	O
boycott	O
british	B-MISC
lamb	O
.	O


In [9]:
for token, label in zip(valid_token_seq[0], valid_label_seq[0]):
    print(f"{token}\t{label}")

cricket	O
-	O
leicestershire	B-ORG
take	O
over	O
at	O
top	O
after	O
innings	O
victory	O
.	O


In [10]:
for token, label in zip(test_token_seq[0], test_label_seq[0]):
    print(f"{token}\t{label}")

soccer	O
-	O
japan	B-LOC
get	O
lucky	O
win	O
,	O
china	B-PER
in	O
surprise	O
defeat	O
.	O


In [11]:
assert len(train_token_seq) == len(train_label_seq), "Длины тренировочных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert len(valid_token_seq) == len(valid_label_seq), "Длины валидационных token_seq и label_seq не совпадают, ошибка в функции read_conll2003"
assert len(test_token_seq) == len(test_label_seq), "Длины тестовых token_seq и label_seq не совпадают, ошибка в функции read_conll2003"

assert train_token_seq[0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Ошибка в тренировочном token_seq"
assert train_label_seq[0] == ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O'], "Ошибка в тренировочном label_seq"

assert valid_token_seq[0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Ошибка в валидационном token_seq"
assert valid_label_seq[0] == ['O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], "Ошибка в валидационном label_seq"

assert test_token_seq[0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Ошибка в тестовом token_seq"
assert test_label_seq[0] == ['O', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O'], "Ошибка в тестовом label_seq"

print("Тесты пройдены!")

Тесты пройдены!


Датасет CoNLL-2003 представлен в виде разметки **BIO**, где лейбл:
- *B-{label}* - начало сущности *{label}*
- *I-{label}* - продолжение сущности *{label}*
- *O* - отсутсвие сущности

Также существует другие разметки последовательностей, например **BILUO**. Подробнее с разметками можно ознакомится во вспомогательном ноутбуке.

### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения:
- {**token**}→{**token_idx**}: соответствие между словом / токеном и строкой в *embedding* матрице (начинается с 0);
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0);

Теперь нам необходимо реализовать две функции:
- get_token2idx
- get_label2idx

которые будут возвращать соответствующие словари.

P.S. token2idx словарь должен также содержать специальные токены:
- `<PAD>` - спецтокен для паддинга, так как мы собираемся обучать модели батчами
- `<UNK>` - спецтокен для обработки слов / токенов, которых нет в словаре (актуально для инференса)

Давайте для удобства дадим им idx 0 и 1 соответственно.

P.P.S. В get_token2idx можно также добавить параметр *min_count*, который будет включать только слова превышающие определенную частоту.

Сначала соберем:
- token2cnt - словарь из уникального слова / токена в количество это слова / токена в тренировочной выборке (важно, что только в тренировочной!)
- label_set - список из уникальных тегов

P.S. Также можно использовать стемминг для того, чтобы преобразовывать разные словоформы одного слова в один токен, но мы опустим этот момент.

**Задание. Реализуйте функции get_token2idx и get_label2idx.** **<font color='red'>(1 балл)</font>**

In [12]:
token2cnt = Counter([token for sentence in train_token_seq for token in sentence])

In [13]:
token2cnt.most_common(10)

[('the', 8390),
 ('.', 7374),
 (',', 7290),
 ('of', 3815),
 ('in', 3621),
 ('to', 3424),
 ('a', 3199),
 ('and', 2872),
 ('(', 2861),
 (')', 2861)]

In [14]:
print(f"Количество уникальных слов в тренировочном датасете: {len(token2cnt)}")
print(f"Количество слов встречающихся только один раз в тренировочном датасете: {len([token for token, cnt in token2cnt.items() if cnt == 1])}")

Количество уникальных слов в тренировочном датасете: 21010
Количество слов встречающихся только один раз в тренировочном датасете: 10060


Как мы видим, у нас есть много слов, которые встречаются только один раз в датасете. Очевидно, что выучиться по ним у нас не получиться, мы только переобучимся, поэтому давайте выкинем такие слова при формировании нашего словаря.

In [15]:
# используйте параметр min_count для того, чтобы отсекать слова частотой cnt < min_count

def get_token2idx(
    token2cnt: Dict[str, int],
    min_count: int,
) -> Dict[str, int]:
    """
    Get mapping from tokens to indices to use with Embedding layer.
    """

    token2cnt_ = dict(filter(lambda x: x[1] >= min_count, token2cnt.items()))
    token2idx: Dict[str, int] = {token: i for i, token in enumerate(['<PAD>', '<UNK>', *token2cnt_.keys()])}

    # YOUR CODE HERE

    return token2idx

In [16]:
token2idx = get_token2idx(token2cnt, min_count=2)

In [17]:
# Функция для сортировки тегов, чтобы сначала был тег O, потом теги B- и только после теги I- (можно задать вручную)

def sort_labels_func(x: str) -> int:
    if x == "O":
        return 0
    elif x.startswith("B-"):
        return 1
    else:
        return 2

label_set = sorted(
    set(label for sentence in train_label_seq for label in sentence),
    key=lambda x: (sort_labels_func(x), x),
)

In [18]:
label_set

['O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']

In [19]:
def get_label2idx(label_set: List[str]) -> Dict[str, int]:
    """
    Get mapping from labels to indices.
    """

    label2idx: Dict[str, int] = {lab: i for i, lab in enumerate(label_set)}

    # YOUR CODE HERE

    return label2idx

In [20]:
label2idx = get_label2idx(label_set)

Посмотрим на то, что мы получили:

In [21]:
for token, idx in list(token2idx.items())[:10]:
    print(f"{token}\t{idx}")

<PAD>	0
<UNK>	1
eu	2
german	3
call	4
to	5
boycott	6
british	7
lamb	8
.	9


In [22]:
for label, idx in label2idx.items():
    print(f"{label}\t{idx}")

O	0
B-LOC	1
B-MISC	2
B-ORG	3
B-PER	4
I-LOC	5
I-MISC	6
I-ORG	7
I-PER	8


In [23]:
assert len(get_token2idx(token2cnt, min_count=1)) == 21012, "Ошибка в длине словаря, скорее всего неверно реализован min_count"
assert len(token2idx) == 10952, "Неправильная длина token2idx, скорее всего неверно реализован min_count"
assert len(label2idx) == 9, "Неправильная длина label2idx"

assert list(token2idx.items())[:10] == [('<PAD>', 0), ('<UNK>', 1), ('eu', 2), ('german', 3), ('call', 4), ('to', 5), ('boycott', 6), ('british', 7), ('lamb', 8), ('.', 9)], "Неправильно сформированный token2idx"
assert label2idx == {'O': 0, 'B-LOC': 1, 'B-MISC': 2, 'B-ORG': 3, 'B-PER': 4, 'I-LOC': 5, 'I-MISC': 6, 'I-ORG': 7, 'I-PER': 8}, "Неправильно сформированный label2idx"

print("Тесты пройдены!")

Тесты пройдены!


### Подготовка датасета и загрузчика

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины.

Из предыдущего практического задания вы должны знать о `Dataset`'е (`torch.utils.data.Dataset`) - структура данных, которая хранит и может по индексу отдавать данные для обучения. Датасет должен наследоваться от стандартного PyTorch класса Dataset и переопределять методы `__len__` и `__getitem__`.

Метод `__getitem__` должен возвращать индексированную последовательность и её теги.

**Не забудьте** про `<UNK>` спецтокен для неизвестных слов!
    
Давайте напишем кастомный датасет под нашу задачу, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов
- token2idx
- label2idx

и возвращать из метода `__getitem__` два int64 тензора (`torch.LongTensor`) из индексов слов / токенов в сэмпле и индексов соответвующих тегов:

**Задание. Реализуйте класс датасета NERDataset.** **<font color='red'>(1 балл)</font>**

In [24]:
class NERDataset(torch.utils.data.Dataset):
    """
    PyTorch Dataset for NER.
    """

    def __init__(
        self,
        token_seq: List[List[str]],
        label_seq: List[List[str]],
        token2idx: Dict[str, int],
        label2idx: Dict[str, int],
    ):
        self.token2idx = token2idx
        self.label2idx = label2idx

        self.token_seq = [self.process_tokens(tokens, token2idx) for tokens in token_seq]
        self.label_seq = [self.process_labels(labels, label2idx) for labels in label_seq]

    def __len__(self):
        return len(self.token_seq)

    def __getitem__(
        self,
        idx: int,
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        # YOUR CODE HERE
        
        return self.token_seq[idx], self.label_seq[idx]
    
    @staticmethod
    def process_tokens(
        tokens: List[str],
        token2idx: Dict[str, int],
        unk: str = "<UNK>",
    ) -> List[int]:
        """
        Transform list of tokens into list of tokens' indices.
        """
        # YOUR CODE HERE
        return torch.LongTensor([token2idx[k] if k in token2idx else token2idx[unk] for k in tokens])

    @staticmethod
    def process_labels(
        labels: List[str],
        label2idx: Dict[str, int],
    ) -> List[int]:
        """
        Transform list of labels into list of labels' indices.
        """
        # YOUR CODE HERE
        return torch.LongTensor([label2idx[k] for k in labels])

Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [25]:
train_dataset = NERDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
valid_dataset = NERDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
test_dataset = NERDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)

Посмотрим на то, что мы получили:

In [26]:
train_dataset[0]

[tensor([2, 1, 3, 4, 5, 6, 7, 8, 9]), tensor([3, 0, 2, 0, 0, 0, 2, 0, 0])]

In [27]:
valid_dataset[0]

[tensor([1737,  571, 1777,  197,  687,  145,  349,  111, 1819, 1558,    9]),
 tensor([0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0])]

In [28]:
test_dataset[0]

[tensor([1516,  571, 1434, 1729, 4893, 2014,   67,  310,  215, 3157, 3139,    9]),
 tensor([0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0])]

In [29]:
assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
assert len(test_dataset) == 3683, "Неправильная длина test_dataset"

assert torch.equal(train_dataset[0][0], torch.tensor([2,1,3,4,5,6,7,8,9])), "Неправильно сформированный train_dataset"
assert torch.equal(train_dataset[0][1], torch.tensor([3,0,2,0,0,0,2,0,0])), "Неправильно сформированный train_dataset"

assert torch.equal(valid_dataset[0][0], torch.tensor([1737,571,1777,197,687,145,349,111,1819,1558,9])), "Неправильно сформированный valid_dataset"
assert torch.equal(valid_dataset[0][1], torch.tensor([0,0,3,0,0,0,0,0,0,0,0])), "Неправильно сформированный valid_dataset"

assert torch.equal(test_dataset[0][0], torch.tensor([1516,571,1434,1729,4893,2014,67,310,215,3157,3139,9])), "Неправильно сформированный test_dataset"
assert torch.equal(test_dataset[0][1], torch.tensor([0,0,1,0,0,0,0,4,0,0,0,0])), "Неправильно сформированный test_dataset"

print("Тесты пройдены!")

Тесты пройдены!


Для того, чтобы дополнять последовательности паддингом, будем использовать параметр `collate_fn` класса `DataLoader`.

Принимая последовательность пар тензоров для предложений и тегов, необходимо дополнить все последовательности до последовательности максимальной длины в батче.

Используйте для дополнения спецтокен `<PAD>` для последовательностей слов / токенов и -1 для последовательностей тегов.

**hint**: удобно использовать метод **torch.nn.utils.rnn**. Обратите особое внимание на параметр *batch_first*

`Collator` можно реализовать двумя способами:
- класс с методом `__call__`
- функцию

Мы пойдем первым путем.

Инициализировать экземпляр класса `Collator` (метод `__init__`) с помощью двух параметров:
- id `<PAD>` спецтокена для последовательностей слов / токенов
- id `<PAD>` спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов
    
P.S. `<PAD>` значение нужно для того, чтобы при подсчете лосса легко отличать западдированные токены от других. Можно использовать параметр *ignore_index* при инициализации лосса.

**Задание. Реализуйте класс коллатора NERCollator.** **<font color='red'>(1 балл)</font>**

In [30]:
from torch.nn.utils.rnn import pad_sequence

class NERCollator:
    """
    Collator that handles variable-size sentences.
    """

    def __init__(
        self,
        token_padding_value: int,
        label_padding_value: int,
    ):
        self.token_padding_value = token_padding_value
        self.label_padding_value = label_padding_value

    def __call__(
        self,
        batch: List[Tuple[torch.LongTensor, torch.LongTensor]],
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:

        tokens, labels = zip(*batch)

        # YOUR CODE HERE
        tokens = pad_sequence(tokens, True, self.token_padding_value)
        labels = pad_sequence(labels, True, self.label_padding_value)

        return tokens, labels

In [31]:
collator = NERCollator(
    token_padding_value=token2idx["<PAD>"],
    label_padding_value=-1,
)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [32]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

Посмотрим на то, что мы получили:

In [33]:
tokens, labels = next(iter(train_dataloader))

tokens = tokens.to(device)
labels = labels.to(device)

In [34]:
tokens

tensor([[7796, 1162, 2553, 7237, 1342,    0,    0,    0,    0,    0],
        [ 125, 1167,    1,   67, 1349,  489, 1215, 1364, 1365, 1366]])

In [35]:
labels

tensor([[ 3,  0,  3,  7,  0, -1, -1, -1, -1, -1],
        [ 0,  4,  8,  0,  1,  0,  0,  0,  0,  0]])

In [36]:
train_tokens, train_labels = next(iter(
    torch.utils.data.DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(train_tokens, torch.tensor([[ 2,  1,  3,  4,  5,  6,  7,  8,  9], [10, 11,  0,  0,  0,  0,  0,  0,  0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_labels, torch.tensor([[ 3,  0,  2,  0,  0,  0,  2,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

valid_tokens, valid_labels = next(iter(
    torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(valid_tokens, torch.tensor([[ 1737,   571,  1777,   197,   687,   145,   349,   111,  1819,  1558, 9], [  248, 10679,     0,     0,     0,     0,     0,     0,     0,     0,    0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_labels, torch.tensor([[ 0,  0,  3,  0,  0,  0,  0,  0,  0,  0,  0], [ 1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

test_tokens, test_labels = next(iter(
    torch.utils.data.DataLoader(
        test_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(test_tokens, torch.tensor([[1516,  571, 1434, 1729, 4893, 2014,   67,  310,  215, 3157, 3139,    9], [   1,    1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_labels, torch.tensor([[ 0,  0,  1,  0,  0,  0,  0,  4,  0,  0,  0,  0], [ 4,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

print("Тесты пройдены!")

Тесты пройдены!


## Часть 2. BiLSTM-теггер (6 баллов)

Определите архитектуру сети, используя библиотеку PyTorch. 

Ваша архитектура в этом пункте должна соответствовать стандартному теггеру:
* Embedding слой на входе
* LSTM (однонаправленный или двунаправленный)слой для обработки последовательности
* Dropout (заданный отдельно или встроенный в LSTM) для уменьшения переобучения
* Linear слой на выходе

Для обучения сети используйте поэлементную кросс-энтропийную функцию потерь.

**Обратите внимание**, что `<PAD>` токены не должны учавствовать в подсчёте функции потерь. В качестве оптимизатора рекомендуется использовать Adam. Для получения значений предсказаний по выходам модели используйте функцию `argmax`.

**Задание. Реализуйте класс модели BiLSTM.** **<font color='red'>(2 балл)</font>**

In [37]:
class BiLSTM(torch.nn.Module):
    """
    Bidirectional LSTM architecture.
    """

    def __init__(
        self,
        num_embeddings: int,
        embedding_dim: int,
        hidden_size: int,
        num_layers: int,
        dropout: float,
        bidirectional: bool,
        n_classes: int,
    ):
        super().__init__()
        
        # YOUR CODE HERE
        
        self.embedding = torch.nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = torch.nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional
        )
        self.head = torch.nn.Linear(2 *  hidden_size, n_classes)
        

    def forward(self, tokens: torch.LongTensor) -> torch.Tensor:
        embed = self.embedding(tokens)

        # используем специальную функцию pack_padded_sequence для того, чтобы получить структуру PackedSequence
        # которая не учитывать паддинг при проходе rnn
        length = (tokens != 0).sum(dim=1).detach().cpu()
        packed_embed = torch.nn.utils.rnn.pack_padded_sequence(
            embed, length, batch_first=True, enforce_sorted=False
          )
        
        # используем специальную функцию pad_packed_sequence для того, чтобы получить тензор из PackedSequence
        packed_rnn_output, _ = self.rnn(packed_embed)
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(
            packed_rnn_output, batch_first=True)
        
        
        logits = self.head(rnn_output)
        return logits.transpose(1, 2)

In [38]:
model = BiLSTM(
    num_embeddings=len(token2idx),
    embedding_dim=100,
    hidden_size=100,
    num_layers=1,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
).to(device)

In [39]:
model

BiLSTM(
  (embedding): Embedding(10952, 100)
  (rnn): LSTM(100, 100, bidirectional=True)
  (head): Linear(in_features=200, out_features=9, bias=True)
)

In [40]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

In [41]:
outputs = model(tokens)

In [42]:
assert outputs.shape == torch.Size([2, 9, 10])
assert 2 < criterion(outputs, labels) < 3

print("Тесты пройдены!")

Тесты пройдены!


### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше 0.76. 

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

In [43]:
# создадим SummaryWriter для эксперимента с BiLSTMModel

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/BiLSTMModel")

**Задание. Реализуйте функцию подсчета метрик compute_metrics.** **<font color='red'>(1 балл)</font>**

In [44]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


def compute_metrics(
    outputs: torch.Tensor,
    labels: torch.LongTensor,
) -> Dict[str, float]:
    """
    Compute NER metrics.
    """

    metrics = {}

    # YOUR CODE HERE
    # Не забудюте отфильтровать <PAD> токен

    mask = labels != -1
    y_true = outputs.argmax(1)
#     print(outputs.shape, y_true.shape, labels.shape)
    y_true = y_true[mask]
    y_pred = labels[mask]
    
    # accuracy
    accuracy = accuracy_score(
        y_true=y_true,
        y_pred=y_pred,
    )

    # precision
    precision_micro = precision_score(
        y_true=y_true,
        y_pred=y_pred,
        average="micro",
        zero_division=0,
    )
    precision_macro = precision_score(
        y_true=y_true,
        y_pred=y_pred,
        average="macro",
        zero_division=0,
    )
    precision_weighted = precision_score(
        y_true=y_true,
        y_pred=y_pred,
        average="weighted",
        zero_division=0,
    )

    # recall
    recall_micro = recall_score(
        y_true=y_true,
        y_pred=y_pred,
        average="micro",
        zero_division=0,
        
    )
    recall_macro = recall_score(
        y_true=y_true,
        y_pred=y_pred,
        average="macro",
        zero_division=0,
    )
    recall_weighted = recall_score(
        y_true=y_true,
        y_pred=y_pred,
        average="weighted",
        zero_division=0,
    )

    # f1
    f1_micro = f1_score(
        y_true=y_true,
        y_pred=y_pred,
        average="micro",
        zero_division=0,
    )
    f1_macro = f1_score(
        y_true=y_true,
        y_pred=y_pred,
        average="macro",
        zero_division=0,
    )
    f1_weighted = f1_score(
        y_true=y_true,
        y_pred=y_pred,
        average="weighted",
        zero_division=0,
    )

    metrics["accuracy"] = accuracy

    metrics["precision_micro"]    = precision_micro
    metrics["precision_macro"]    = precision_macro
    metrics["precision_weighted"] = precision_weighted

    metrics["recall_micro"]    = recall_micro
    metrics["recall_macro"]    = recall_macro
    metrics["recall_weighted"] = recall_weighted

    metrics["f1_micro"]    = f1_micro
    metrics["f1_macro"]    = f1_macro
    metrics["f1_weighted"] = f1_weighted

    return metrics

**Задание. Реализуйте функции обучения и тестирования train_epoch и evaluate_epoch.** **<font color='red'>(2 балла)</font>**

In [45]:
def train_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One training cycle (loop).
    """

    model.train()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    for i, (tokens, labels) in tqdm(
        enumerate(dataloader),
        total=len(dataloader),
        desc="loop over train batches",
    ):

        tokens, labels = tokens.to(device), labels.to(device)

        # YOUR CODE HERE
        # Подсчет лосса и шаг оптимизатора
        
        optimizer.zero_grad()
        preds = model(tokens)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        writer.add_scalar(
            "batch loss / train", loss.item(), epoch * len(dataloader) + i
        )

        with torch.no_grad():
            model.eval()
            outputs_inference = model(tokens)
            model.train()

        batch_metrics = compute_metrics(
            outputs=outputs_inference,
            labels=labels,
        )

        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)
            writer.add_scalar(
                f"batch {metric_name} / train",
                metric_value,
                epoch * len(dataloader) + i,
            )

    avg_loss = np.mean(epoch_loss)
    print(f"Train loss: {avg_loss}\n")
    writer.add_scalar("loss / train", avg_loss, epoch)

    for metric_name, metric_value_list in batch_metrics_list.items():
        metric_value = np.mean(metric_value_list)
        print(f"Train {metric_name}: {metric_value}\n")
        writer.add_scalar(f"{metric_name} / train", metric_value, epoch)

In [46]:
def evaluate_epoch(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One evaluation cycle (loop).
    """

    model.eval()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    with torch.no_grad():

        for i, (tokens, labels) in tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            desc="loop over test batches",
        ):

            tokens, labels = tokens.to(device), labels.to(device)

            # YOUR CODE HERE
            # Подсчет лосса
            outputs = model(tokens)
            loss = criterion(outputs, labels)

            epoch_loss.append(loss.item())
            writer.add_scalar(
                "batch loss / test", loss.item(), epoch * len(dataloader) + i
            )

            batch_metrics = compute_metrics(
                outputs=outputs,
                labels=labels,
            )

            for metric_name, metric_value in batch_metrics.items():
                batch_metrics_list[metric_name].append(metric_value)
                writer.add_scalar(
                    f"batch {metric_name} / test",
                    metric_value,
                    epoch * len(dataloader) + i,
                )

        avg_loss = np.mean(epoch_loss)
        print(f"Test loss:  {avg_loss}\n")
        writer.add_scalar("loss / test", avg_loss, epoch)

        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = np.mean(metric_value_list)
            print(f"Test {metric_name}: {metric_value}\n")
            writer.add_scalar(f"{metric_name} / test", np.mean(metric_value), epoch)

In [47]:
def train(
    n_epochs: int,
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
) -> None:
    """
    Training loop.
    """

    for epoch in range(n_epochs):

        print(f"Epoch [{epoch+1} / {n_epochs}]\n")

        train_epoch(
            model=model,
            dataloader=train_dataloader,
            optimizer=optimizer,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
        )
        evaluate_epoch(
            model=model,
            dataloader=test_dataloader,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
        )

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**


In [48]:
# YOUR CODE HERE
model = BiLSTM(
    num_embeddings=len(token2idx),
    embedding_dim=100,
    hidden_size=100,
    num_layers=1,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

train(
    n_epochs=30,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=valid_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
    device=device
)

Epoch [1 / 30]



loop over train batches: 100%|██████████| 7493/7493 [06:10<00:00, 20.25it/s]


Train loss: 0.7131510377905932

Train accuracy: 0.8173141027266931

Train precision_micro: 0.8173141027266931

Train precision_macro: 0.3913753592258223

Train precision_weighted: 0.9778170130800714

Train recall_micro: 0.8173141027266931

Train recall_macro: 0.3592585652464284

Train recall_weighted: 0.8173141027266931

Train f1_micro: 0.8173141027266931

Train f1_macro: 0.36775471367013507

Train f1_weighted: 0.8819794193027808



loop over test batches: 100%|██████████| 3465/3465 [00:47<00:00, 73.26it/s]


Test loss:  0.5533478223042722

Test accuracy: 0.8430531229243555

Test precision_micro: 0.8430531229243555

Test precision_macro: 0.6088866653803825

Test precision_weighted: 0.9557404779464566

Test recall_micro: 0.8430531229243555

Test recall_macro: 0.5770533991568113

Test recall_weighted: 0.8430531229243555

Test f1_micro: 0.8430531229243555

Test f1_macro: 0.585314610336197

Test f1_weighted: 0.8856833228385044

Epoch [2 / 30]



...

Test loss:  0.31281324095697816

Test accuracy: 0.9379809683000903

Test precision_micro: 0.9379809683000903

Test precision_macro: 0.8325358628273329

Test precision_weighted: 0.9434787282723331

Test recall_micro: 0.9379809683000903

Test recall_macro: 0.8342611857161555

Test recall_weighted: 0.9379809683000903

Test f1_micro: 0.9379809683000903

Test f1_macro: 0.8283552499519703

Test f1_weighted: 0.9376130868625075

Epoch [30 / 30]



loop over train batches: 100%|██████████| 7493/7493 [08:00<00:00, 15.59it/s]


Train loss: 0.004098075840617447

Train accuracy: 0.9991591995101557

Train precision_micro: 0.9991591995101557

Train precision_macro: 0.9977816206030162

Train precision_weighted: 0.9993579509441878

Train recall_micro: 0.9991591995101557

Train recall_macro: 0.9978472290574252

Train recall_weighted: 0.9991591995101557

Train f1_micro: 0.9991591995101557

Train f1_macro: 0.9977022063673788

Train f1_weighted: 0.9991820450329616



loop over test batches: 100%|██████████| 3465/3465 [00:57<00:00, 60.60it/s]


Test loss:  0.30422460605292473

Test accuracy: 0.9471245023226331

Test precision_micro: 0.9471245023226331

Test precision_macro: 0.8508823561445538

Test precision_weighted: 0.9611473649400771

Test recall_micro: 0.9471245023226331

Test recall_macro: 0.8525939061210906

Test recall_weighted: 0.9471245023226331

Test f1_micro: 0.9471245023226331

Test f1_macro: 0.8470735776475032

Test f1_weighted: 0.9512233436853534



In [54]:
# тестируем модель

evaluate_epoch(
    model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=0,
)

loop over test batches: 100%|██████████| 3683/3683 [00:49<00:00, 74.70it/s]

Test loss:  0.483811484441782

Test accuracy: 0.9110213117733795

Test precision_micro: 0.9110213117733795

Test precision_macro: 0.7901290933981644

Test precision_weighted: 0.9258598983819958

Test recall_micro: 0.9110213117733795

Test recall_macro: 0.7927784219277771

Test recall_weighted: 0.9110213117733795

Test f1_micro: 0.9110213117733795

Test f1_macro: 0.7859816055043567

Test f1_weighted: 0.9138606485010199



Получилось:  
    Train f1_macro: 0.9977022063673788  
    Valid f1_macro: 0.8470735776475032  
    Test f1_macro: 0.7859816055043567  
Переобучились конечно, но и так сойдёт

## Часть 3. Transformers-теггер (6 баллов)

В данной части задания нужно сделать все то же самое, но с использованием модели на базе архитектуры Transformer, а именно предлагается дообучать предобученную модель **BERT**.

Для данной модели подразумевается специальная подготовка данных, с чего мы и начнем:

Модель **BERT** использует специальный токенизатор WordPiece для разбиения предложений на токены. Готовая предобученная версия такого токенизатора существует в библиотеке **transformers**. Есть два класса: `BertTokenizer` и `BertTokenizerFast`. Использовать можно любой, но второй вариант работает существенно быстрее.

Токенизаторы можно обучать с нуля на своем корпусе данных, а можно подгружать уже готовые. Готовые токенизаторы, как правило, соответствуют предобученной конфигурации модели, которая использует словарь из этого токенизатора. 

Мы будем использовать базовую конфигурацию предобученного **BERT** для модели и токенизатора.

P.S. Часто приходится проводить эксперименты с моделями разной архитектуры, например **BERT** и **GPT**, поэтому удобно использовать класс `AutoTokenizer`, который по названию модели сам определит, какой класс нужен для инициализации токенизатора.

In [56]:
from transformers import AutoTokenizer

In [57]:
model_name = "distilbert-base-cased"

Подгружение предобученных моделей и токенизаторов в **huggingface** происходит с помощью конструктора **from_pretrained**.

В данном конструкторе можно указать либо путь к предобученному токенизатору, либо название предобученной конфигурации, как в нашем случае: тогда **transformers** сам подгрузит нужные параметры:

In [58]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

### Подготовка словарей

В сравнении с рекуррентными моделями, на больше не нужно заниматься сборкой словаря, так как это уже сделано заранее благодаря токенизаторам и алгоритмам, стоящими за ними.

Но нам как и прежде потребуется:
- {**label**}→{**label_idx**}: соответствие между тегом и уникальным индексом (начинается с 0);

Но данное отображение у нас уже реализовано в одной из предыдущих частей задания.

### Подготовка датасета и загрузчика

Мы также хотим обучать модель батчами, поэтому нам как и прежде понадобятся `Dataset`, `Collator` и `DataLoader`.

Но мы не можем переиспользовать те, что в предыдущих частях задания, так как обработка данных должна производится немного иначе с использованием токенизатора.

Давайте напишем новый кастомный датасет, который на вход (метод `__init__`) будет принимать:
- token_seq - список списков слов / токенов
- label_seq - список списков тегов

и возвращать из метода `__getitem__` два списка:
- список текстовых значений (`List[str]`) из индексов токенов в сэмпле
- список целочисленных значений (`List[int]`) из индексов соответвующих тегов

P.S. В отличие от предыдущего кастомного датасет, здесь мы возвращаем два `List`'а вместо `torch.LongTensor`, так как логику формирования западдированного батча мы перенесем в `Collator` из-за специфики работы токенизатора - он сам возвращает уже западдированный тензор с индексами токенов, а для индексов тегов нам нужно будет сделать это самостоятельно по аналогии с предыдущим датасетом.

**Задание. Реализуйте класс датасета TransformersDataset.** **<font color='red'>(1 балл)</font>**

In [59]:
class TransformersDataset(torch.utils.data.Dataset):
    """
    Transformers Dataset for NER.
    """

    def __init__(
        self,
        token_seq: List[List[str]],
        label_seq: List[List[str]],
    ):
        self.token_seq = token_seq
        self.label_seq = [self.process_labels(labels, label2idx) for labels in label_seq]

    def __len__(self):
        return len(self.token_seq)

    def __getitem__(
        self,
        idx: int,
    ) -> Tuple[List[str], List[int]]:
        # YOUR CODE HERE
        return self.token_seq[idx], self.label_seq[idx]
    
    @staticmethod
    def process_labels(
        labels: List[str],
        label2idx: Dict[str, int],
    ) -> List[int]:
        """
        Transform list of labels into list of labels' indices.
        """
        # YOUR CODE HERE
        return [label2idx[k] for k in labels]

Создадим три датасета:
- *train_dataset*
- *valid_dataset*
- *test_dataset*

In [143]:
train_dataset = TransformersDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
)
valid_dataset = TransformersDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
)
test_dataset = TransformersDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
)

Посмотрим на то, что мы получили:

In [144]:
train_dataset[0]

(['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'],
 [3, 0, 2, 0, 0, 0, 2, 0, 0])

In [145]:
valid_dataset[0]

(['cricket',
  '-',
  'leicestershire',
  'take',
  'over',
  'at',
  'top',
  'after',
  'innings',
  'victory',
  '.'],
 [0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0])

In [146]:
test_dataset[0]

(['soccer',
  '-',
  'japan',
  'get',
  'lucky',
  'win',
  ',',
  'china',
  'in',
  'surprise',
  'defeat',
  '.'],
 [0, 0, 1, 0, 0, 0, 0, 4, 0, 0, 0, 0])

In [147]:
assert len(train_dataset) == 14986, "Неправильная длина train_dataset"
assert len(valid_dataset) == 3465, "Неправильная длина valid_dataset"
assert len(test_dataset) == 3683, "Неправильная длина test_dataset"

assert train_dataset[0][0] == ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.'], "Неправильно сформированный train_dataset"
assert train_dataset[0][1] == [3,0,2,0,0,0,2,0,0], "Неправильно сформированный train_dataset"

assert valid_dataset[0][0] == ['cricket', '-', 'leicestershire', 'take', 'over', 'at', 'top', 'after', 'innings', 'victory', '.'], "Неправильно сформированный valid_dataset"
assert valid_dataset[0][1] == [0,0,3,0,0,0,0,0,0,0,0], "Неправильно сформированный valid_dataset"

assert test_dataset[0][0] == ['soccer', '-', 'japan', 'get', 'lucky', 'win', ',', 'china', 'in', 'surprise', 'defeat', '.'], "Неправильно сформированный test_dataset"
assert test_dataset[0][1] == [0,0,1,0,0,0,0,4,0,0,0,0], "Неправильно сформированный test_dataset"

print("Тесты пройдены!")

Тесты пройдены!


Реализуем новый `Collator`.

Инициализировать коллатор будет 3 аргументами:
- токенизатор
- параметры токенизатора в виде словаря (затем используем как `**kwargs`)
- id спецтокена для последовательностей тегов (значение -1)

Метод `__call__` на вход принимает батч, а именно список кортежей того, что нам возвращается из датасета. В нашем случае это список кортежей двух int64 тензоров - `List[Tuple[torch.LongTensor, torch.LongTensor]]`.

На выходе мы хотим получить два тензора:
- западденные индексы слов / токенов
- западденные индексы тегов

**Задание. Реализуйте класс коллатора TransformersCollator.** **<font color='red'>(2 балла)</font>**

In [262]:
from transformers import PreTrainedTokenizer
from transformers.tokenization_utils_base import BatchEncoding


class TransformersCollator:
    """
    Transformers Collator that handles variable-size sentences.
    """

    def __init__(
        self,
        tokenizer: PreTrainedTokenizer,
        tokenizer_kwargs: Dict[str, Any],
        label_padding_value: int,
    ):
        self.tokenizer = tokenizer
        self.tokenizer_kwargs = tokenizer_kwargs
        
        self.label_padding_value = label_padding_value

    def __call__(
        self,
        batch: List[Tuple[List[str], List[int]]],
    ) -> Tuple[torch.LongTensor, torch.LongTensor]:
        tokens, labels = zip(*batch)

        # YOUR CODE HERE
        
        tokens = self.tokenizer(list(tokens), **self.tokenizer_kwargs)
        labels = self.encode_labels(tokens, labels, self.label_padding_value)
        tokens.pop("offset_mapping")

        return tokens, labels
    
    @staticmethod
    def encode_labels(
        tokens: BatchEncoding,
        labels: List[List[int]],
        label_padding_value: int,
    ) -> torch.LongTensor:

        encoded_labels = []

        for doc_labels, doc_offset in zip(labels, tokens.offset_mapping):

            doc_enc_labels = np.ones(len(doc_offset), dtype=int) * label_padding_value
            arr_offset = np.array(doc_offset)

            doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
            encoded_labels.append(doc_enc_labels.tolist())

        return torch.LongTensor(encoded_labels)

In [241]:
tokenizer_kwargs = {
    "is_split_into_words":    True,
    "return_offsets_mapping": True,
    "padding":                True,
    "truncation":             True,
    "max_length":             512,
    "return_tensors":         "pt",
}

In [242]:
collator = TransformersCollator(
    tokenizer=tokenizer,
    tokenizer_kwargs=tokenizer_kwargs,
    label_padding_value=-1,
)

Теперь всё готово, чтобы задать `DataLoader`'ы:

In [243]:
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=2,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

Посмотрим на то, что мы получили:

In [244]:
tokens, labels = next(iter(train_dataloader))

tokens = tokens.to(device)
labels = labels.to(device)

In [245]:
tokens

{'input_ids': tensor([[  101, 14247,  1548,  1820,   118,  4775,   118,  1572,   102],
        [  101,   118,  1202,  6063,  6817,  1204,   118,   102,     0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 0]])}

In [246]:
labels

tensor([[-1,  1, -1,  0, -1, -1, -1, -1, -1],
        [-1,  0, -1, -1, -1, -1, -1, -1, -1]])

In [247]:
train_tokens, train_labels = next(iter(
    torch.utils.data.DataLoader(
        train_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(train_tokens['input_ids'], torch.tensor([[  101,   174,  1358, 22961,   176, 14170,  1840,  1106, 21423,  9304, 10721,  1324,  2495, 12913,   119,   102], [  101, 11109,  1200,  1602,  6715,   102,     0,     0,     0,     0,    0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(train_labels, torch.tensor([[-1,  3, -1,  0,  2, -1,  0,  0,  0,  2, -1, -1,  0, -1,  0, -1], [-1,  4, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

valid_tokens, valid_labels = next(iter(
    torch.utils.data.DataLoader(
        valid_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(valid_tokens['input_ids'], torch.tensor([[  101,  5428,   118,  5837, 18117,  5759, 15189,  1321,  1166,  1120,  1499,  1170,  6687,  2681,   119,   102], [  101, 25338, 17996,  1820,   118,  4775,   118,  1476,   102,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(valid_labels, torch.tensor([[-1,  0,  0,  3, -1, -1, -1,  0,  0,  0,  0,  0,  0,  0,  0, -1], [-1,  1, -1,  0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

test_tokens, test_labels = next(iter(
    torch.utils.data.DataLoader(
        test_dataset,
        batch_size=2,
        shuffle=False,
        collate_fn=collator,
    )
))
assert torch.equal(test_tokens['input_ids'], torch.tensor([[  101,  5862,   118,   179, 26519,  1179,  1243,  6918,  1782,   117,  5144,  1161,  1107,  3774,  3326,   119,   102], [  101,  9468,  3309,  1306, 19122,  2293,   102,     0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_tokens['attention_mask'], torch.tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])), "Похоже на ошибку в коллаторе"
assert torch.equal(test_labels, torch.tensor([[-1,  0,  0,  1, -1, -1,  0,  0,  0,  0,  4, -1,  0,  0,  0,  0, -1], [-1,  4, -1, -1,  8, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]])), "Похоже на ошибку в коллаторе"

print("Тесты пройдены!")

Тесты пройдены!


В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Для задачи NER будем использовать класс `BertForTokenClassification`.

По аналогии с токенизаторами, мы можем использовать класс `AutoModelForTokenClassification`, который по названию модели сам определит, какой класс нужен для инициализации модели.

In [248]:
from transformers import AutoModelForTokenClassification

In [249]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label2idx),
).to(device)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

In [250]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [251]:
outputs = model(**tokens)

In [252]:
assert 2 < criterion(outputs["logits"].transpose(1, 2), labels) < 3

print("Тесты пройдены!")

Тесты пройдены!


In [253]:
# создадим SummaryWriter для эксперимента с BiLSTMModel

from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter(log_dir=f"logs/Transformer")

### Эксперименты

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1-macro мере на валидационной и тестовой выборках было не меньше 0.9. 

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

Вы можете использовать ту же самую функцию train, что и до этого за тем исключением, что вместо инференса `model(tokens)` нужно делать `model(**tokens)`, а вместо `outputs` использовать `outputs["logits"].transpose(1, 2)`

**Задание. Проведите эксперименты.** **<font color='red'>(2 балла)</font>**


In [254]:
# YOUR CODE HERE

def train_epoch_1(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One training cycle (loop).
    """

    model.train()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    for i, (tokens, labels) in tqdm(
        enumerate(dataloader),
        total=len(dataloader),
        desc="loop over train batches",
    ):

        tokens, labels = tokens.to(device), labels.to(device)

        # YOUR CODE HERE
        # Подсчет лосса и шаг оптимизатора
        
        optimizer.zero_grad()
        preds = model(**tokens)["logits"].transpose(1, 2)
        loss = criterion(preds, labels)
        loss.backward()
        optimizer.step()

        epoch_loss.append(loss.item())
        writer.add_scalar(
            "batch loss / train", loss.item(), epoch * len(dataloader) + i
        )

        with torch.no_grad():
            model.eval()
            outputs_inference = model(**tokens)["logits"].transpose(1, 2)
            model.train()

        batch_metrics = compute_metrics(
            outputs=outputs_inference,
            labels=labels,
        )

        for metric_name, metric_value in batch_metrics.items():
            batch_metrics_list[metric_name].append(metric_value)
            writer.add_scalar(
                f"batch {metric_name} / train",
                metric_value,
                epoch * len(dataloader) + i,
            )

    avg_loss = np.mean(epoch_loss)
    print(f"Train loss: {avg_loss}\n")
    writer.add_scalar("loss / train", avg_loss, epoch)

    for metric_name, metric_value_list in batch_metrics_list.items():
        metric_value = np.mean(metric_value_list)
        print(f"Train {metric_name}: {metric_value}\n")
        writer.add_scalar(f"{metric_name} / train", metric_value, epoch)
        
def evaluate_epoch_1(
    model: torch.nn.Module,
    dataloader: torch.utils.data.DataLoader,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
    epoch: int,
) -> None:
    """
    One evaluation cycle (loop).
    """

    model.eval()

    epoch_loss = []
    batch_metrics_list = defaultdict(list)

    with torch.no_grad():

        for i, (tokens, labels) in tqdm(
            enumerate(dataloader),
            total=len(dataloader),
            desc="loop over test batches",
        ):

            tokens, labels = tokens.to(device), labels.to(device)

            # YOUR CODE HERE
            # Подсчет лосса
            outputs = model(**tokens)["logits"].transpose(1, 2)
            loss = criterion(outputs, labels)

            epoch_loss.append(loss.item())
            writer.add_scalar(
                "batch loss / test", loss.item(), epoch * len(dataloader) + i
            )

            batch_metrics = compute_metrics(
                outputs=outputs,
                labels=labels,
            )

            for metric_name, metric_value in batch_metrics.items():
                batch_metrics_list[metric_name].append(metric_value)
                writer.add_scalar(
                    f"batch {metric_name} / test",
                    metric_value,
                    epoch * len(dataloader) + i,
                )

        avg_loss = np.mean(epoch_loss)
        print(f"Test loss:  {avg_loss}\n")
        writer.add_scalar("loss / test", avg_loss, epoch)

        for metric_name, metric_value_list in batch_metrics_list.items():
            metric_value = np.mean(metric_value_list)
            print(f"Test {metric_name}: {metric_value}\n")
            writer.add_scalar(f"{metric_name} / test", np.mean(metric_value), epoch)        
            
def train_1(
    n_epochs: int,
    model: torch.nn.Module,
    train_dataloader: torch.utils.data.DataLoader,
    test_dataloader: torch.utils.data.DataLoader,
    optimizer: torch.optim.Optimizer,
    criterion: torch.nn.Module,
    writer: SummaryWriter,
    device: torch.device,
) -> None:
    """
    Training loop.
    """

    for epoch in range(n_epochs):

        print(f"Epoch [{epoch+1} / {n_epochs}]\n")

        train_epoch_1(
            model=model,
            dataloader=train_dataloader,
            optimizer=optimizer,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
        )
        evaluate_epoch_1(
            model=model,
            dataloader=test_dataloader,
            criterion=criterion,
            writer=writer,
            device=device,
            epoch=epoch,
        )

In [255]:
# YOUR CODE HERE
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=len(label2idx),
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

train_1(
    n_epochs=10,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=valid_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
    device=device
)

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

Epoch [1 / 10]



loop over train batches:   3%|▎         | 212/7493 [03:07<1:47:24,  1.13it/s]



Модель хорошая, гиперпараметры чувствует, то переобучается, то не переобучается, эксперименты были проделаны мысленно, потому что гонять модельки не было времени и желания :(

## Часть 4 - Бонус. BiLSTMAttention-теггер (2 баллa)

Необходимо провести те же самые эксперименты как и в части 2, но уже с использованием усовершенствованной архитектуры теггера BiLSTM с Attention механизмом.

**Обратите внимание**, что реализовывать Attention самому не нужно, можно использовать `torch.nn.MultiheadAttention`.

Также сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров и проведите небольшой сравнительный анализ с предыдущей архитектурой. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

**Задание. Реализуйте класс модели BiLSTMAttn.** **<font color='red'>(1 балл)</font>**

In [277]:
# YOUR CODE HERE
class BiLSTMAttn(torch.nn.Module):

    def __init__(
        self,
        num_embeddings: int,
        embedding_dim: int,
        hidden_size: int,
        num_heads: int, 
        num_layers: int,
        dropout: float,
        bidirectional: bool,
        n_classes: int,
    ):
        super().__init__()
        
        # YOUR CODE HERE
        self.hidden_size = hidden_size
        self.embedding = torch.nn.Embedding(num_embeddings, embedding_dim)
        self.rnn = torch.nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            dropout=dropout,
            bidirectional=bidirectional
        )
        self.attention = torch.nn.MultiheadAttention(hidden_size, num_heads)
        self.head = torch.nn.Linear(hidden_size, n_classes)
        

    def forward(self, tokens: torch.LongTensor) -> torch.Tensor:
        embed = self.embedding(tokens)

        length = (tokens != 0).sum(dim=1).detach().cpu()
        packed_embed = torch.nn.utils.rnn.pack_padded_sequence(embed, length, batch_first=True, enforce_sorted=False)
        
        packed_rnn_output, _ = self.rnn(packed_embed)
        rnn_output, _ = torch.nn.utils.rnn.pad_packed_sequence(packed_rnn_output, batch_first=True)
        
        # (batch_size, word_pad_len, hidden_size)
        H = rnn_output[ :, :, : self.hidden_size] + rnn_output[ :, :, self.hidden_size : ]
        
        # (batch_size, hidden_size), (batch_size, word_pad_len)
        attn_output, weights = self.attention(H, H, H)
        
        logits = self.head(attn_output)
        return logits.transpose(1, 2)

**Задание. Проведите эксперименты и побейте метрику из части 2.** **<font color='red'>(1 балл)</font>**

P.S. Eсли качества увеличить не получилось, это нужно обосновать

In [278]:
train_dataset = NERDataset(
    token_seq=train_token_seq,
    label_seq=train_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
valid_dataset = NERDataset(
    token_seq=valid_token_seq,
    label_seq=valid_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)
test_dataset = NERDataset(
    token_seq=test_token_seq,
    label_seq=test_label_seq,
    token2idx=token2idx,
    label2idx=label2idx,
)

collator = NERCollator(
    token_padding_value=token2idx["<PAD>"],
    label_padding_value=-1,
)

train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collator,
)
valid_dataloader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset,
    
    batch_size=1,  # для корректных замеров метрик оставить batch_size=1
    shuffle=False, # для корректных замеров метрик оставить shuffle=False
    collate_fn=collator,
)

In [279]:
writer = SummaryWriter(log_dir=f"logs/BiLSTMAttn")

In [280]:
# YOUR CODE HERE

model = BiLSTMAttn(
    num_embeddings=len(token2idx),
    embedding_dim=100,
    hidden_size=100,
    num_heads=5,
    num_layers=1,
    dropout=0.0,
    bidirectional=True,
    n_classes=len(label2idx),
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
criterion = torch.nn.CrossEntropyLoss(ignore_index=-1)

train(
    n_epochs=30,
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=valid_dataloader,
    optimizer=optimizer,
    criterion=criterion,
    writer=writer,
    device=device
)

Epoch [1 / 30]



loop over train batches: 100%|██████████| 235/235 [01:09<00:00,  3.39it/s]


Train loss: 0.44895218636127227

Train accuracy: 0.89549766613797

Train precision_micro: 0.89549766613797

Train precision_macro: 0.4130295347534538

Train precision_weighted: 0.9586554377237648

Train recall_micro: 0.89549766613797

Train recall_macro: 0.48607917571015585

Train recall_weighted: 0.89549766613797

Train f1_micro: 0.89549766613797

Train f1_macro: 0.4268072358595154

Train f1_weighted: 0.9235846906451642



loop over test batches: 100%|██████████| 3465/3465 [00:55<00:00, 62.37it/s]


Test loss:  1.1141752944095396

Test accuracy: 0.6614308162608106

Test precision_micro: 0.6614308162608106

Test precision_macro: 0.3605826479679185

Test precision_weighted: 0.6437454565007791

Test recall_micro: 0.6614308162608106

Test recall_macro: 0.3540742947950307

Test recall_weighted: 0.6614308162608106

Test f1_micro: 0.6614308162608106

Test f1_macro: 0.3447216124168545

Test f1_weighted: 0.6384460381027138

Epoch [2 / 30]



...

Test loss:  0.2667888026879212

Test accuracy: 0.9413158634352684

Test precision_micro: 0.9413158634352684

Test precision_macro: 0.8310082626960857

Test precision_weighted: 0.9583809300167463

Test recall_micro: 0.9413158634352684

Test recall_macro: 0.8335315279920728

Test recall_weighted: 0.9413158634352684

Test f1_micro: 0.9413158634352684

Test f1_macro: 0.827011286705762

Test f1_weighted: 0.9461432955198583

Epoch [30 / 30]



loop over train batches: 100%|██████████| 235/235 [01:17<00:00,  3.03it/s]


Train loss: 0.038044986166456275

Train accuracy: 0.9959423631637833

Train precision_micro: 0.9959423631637833

Train precision_macro: 0.9792950208923815

Train precision_weighted: 0.9961754153368438

Train recall_micro: 0.9959423631637833

Train recall_macro: 0.9838461366888279

Train recall_weighted: 0.9959423631637833

Train f1_micro: 0.9959423631637833

Train f1_macro: 0.9802463270274956

Train f1_weighted: 0.9959665426510738



loop over test batches: 100%|██████████| 3465/3465 [00:51<00:00, 67.02it/s]

Test loss:  0.24866064922778194

Test accuracy: 0.9442938754502808

Test precision_micro: 0.9442938754502808

Test precision_macro: 0.8435509775170716

Test precision_weighted: 0.9603001539862639

Test recall_micro: 0.9442938754502808

Test recall_macro: 0.8452238857581129

Test recall_weighted: 0.9442938754502808

Test f1_micro: 0.9442938754502808

Test f1_macro: 0.8388774400147997

Test f1_weighted: 0.9484623429486158



In [281]:
# тестируем модель

evaluate_epoch(
    model=model,
    dataloader=test_dataloader,
    criterion=criterion,
    writer=writer,
    device=device,
    epoch=0,
)

loop over test batches: 100%|██████████| 3683/3683 [00:50<00:00, 73.21it/s]

Test loss:  0.3860595983688913

Test accuracy: 0.913907124077706

Test precision_micro: 0.913907124077706

Test precision_macro: 0.7930594455668198

Test precision_weighted: 0.9348268606500798

Test recall_micro: 0.913907124077706

Test recall_macro: 0.792615705018836

Test recall_weighted: 0.913907124077706

Test f1_micro: 0.913907124077706

Test f1_macro: 0.7869571943151509

Test f1_weighted: 0.9194013660322837



Получилось:  
    Train f1_macro: 0.9802463270274956  
    Valid f1_macro: 0.8388774400147997  
    Test f1_macro: 0.7869571943151509  
Плюс минус то же самое и вышло, чтоб получить качество лучше, надо делать эксперименты и для той, и для этой модели, но мне лень поэтому вот такие вот результаты!   

Надеюсь, это хорошее обоснование :)